In [182]:
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
from time import time
from math import ceil
from datetime import timedelta
import unicodedata  

In [194]:
#documentating information about the excel files and sheets inside of it.

files = os.listdir()

xslx_files = [x for x in files if x.split('.')[-1]=='xlsx' and x.split('.')[0]!='BI_Financeiro']

def get_sheets_names(x):
    xlsx = pd.ExcelFile(x)
    return xlsx.sheet_names  # see all sheet names

sheets_names = [get_sheets_names(excel) for excel in xslx_files ]

lst_src = list(zip(xslx_files, sheets_names))

for i,v in enumerate(lst_src): 
    exec(f"xlsx_{i} = '{v[0]}'")

df_xlsx = pd.DataFrame(lst_src, columns=['xlsx_file','sheets'])

df_xlsx.to_csv('df_xlsx.csv')


print('df_xlsx: ')
df_xlsx


df_xlsx: 


,xlsx_file,sheets
0,Controle de Empenhos e NC 2022.xlsx,"[ControleGeral, SintéticoGeral, SintéticoNC, A..."
1,Controle de NC 2022.xlsx,"[SintéticoNC, AnalíticoNC, ÍndiceCorreção]"
2,Proponentes.xlsx,[Proponentes]


In [195]:
start_time = time()


df_nm =[
('df_an', 'AnalíticoNC'),
('df_sn', 'SintéticoNC'), 
('df_ic', 'ÍndiceCorreção'),
('df_pr', 'Proponentes'),
('df_cg', 'ControleGeral')]

df_sheets = pd.DataFrame(df_nm, columns = ['df_name', 'sheet_name'])

e1 = 'Controle de NC 2022.xlsx'
e2 = 'Proponentes.xlsx'
e3 = 'Controle de Empenhos e NC 2022.xlsx'   

#filter_xlsx = (df_xlsx[''] ==) 

df_an = pd.read_excel(e1,sheet_name=df_nm[0][1],
usecols="B:J", skiprows=1,parse_dates=True)

df_sn = pd.read_excel(e1, sheet_name=df_nm[1][1],
usecols="B:J", skiprows=3,parse_dates=True)

df_ic = pd.read_excel(e1, sheet_name=df_nm[2][1],
skiprows=1,usecols="B:E")

df_pr = pd.read_excel(e2, sheet_name=0)

df_cg = pd.read_excel(e3, sheet_name=df_nm[4][1]
, skiprows=12,usecols="c:w")

##----------------------------------------------------------
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time
df_sheets

/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


Time: This code took 2 seconds to run.


,df_name,sheet_name
0,df_an,AnalíticoNC
1,df_sn,SintéticoNC
2,df_ic,ÍndiceCorreção
3,df_pr,Proponentes
4,df_cg,ControleGeral


In [185]:
start_time = time()
# criando função 'regexing' p/ subst. nomes abreviados de forma a normalizar as strings
def regexing(s):
    lst = [
    ('PRF ','POLICIA RODOVIARIA FEDERAL '),
    ('DEP. FEDERAL ', ''),
    ('MCOM ', 'MINISTERIO DAS COMUNICACOES '),
    ('DEP. FEDERAL', ''),
    ('PRF ','POLICIA RODOVIARIA FEDERAL '),
    ('MIN. ','MINISTERIO '),
    ('SENADORA ',''),
    ('SENADOR ',''),
    ]
    for string in lst:
        return re.sub(string[0],string[1],s)

lst_proponente = []

for index, prop in enumerate(df_pr['prop_pk1']):
    if df_pr['orgao'][index] == "senado":
        lst_proponente.append('SEN. ' + prop)
    elif  df_pr['orgao'][index] == "camara":
        lst_proponente.append('DEP. ' + prop)
    else: 
        lst_proponente.append(prop)

df_pr['prop_pk2'] = lst_proponente

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This ckode took 1 seconds to run.

Time: This code took 1 seconds to run.


In [196]:
start_time = time()

def strip_accents(cln):
   cln = str(cln)
   return ''.join(c for c in unicodedata.normalize('NFD', cln)if unicodedata.category(c) != 'Mn')

def clean_column_name(cln):
    return strip_accents(cln.strip('[]()').lower().replace(' ','_'))

def rename_df(df):
    new_cln = [clean_column_name(cln) for cln in df.columns.values]
    old_cln = [cln for cln in df.columns.values]
    dct = dict(zip(old_cln, new_cln))
    df.rename(columns=dct,inplace=True)
    lst_df_clns.append(new_cln)
    return df

#creating a list with all dfs from the data dictionary
dfs = [ x for x,y in df_nm]
cmd = str(dfs).replace("'","") 
exec('dfs = {}'.format(cmd))

lst_df_clns = []
for df in dfs:
    rename_df(df)

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time
df_nm = list(zip(df_nm,lst_df_clns))

lst_df_clns

Time: This code took 1 seconds to run.


[['proponente',
  'nota_de_credito',
  'programa',
  'mes',
  'valor_faturado',
  'valor_economizado',
  'valor_utilizado',
  'saldo_no_periodo',
  'relatorio'],
 ['nc',
  'proponente',
  'programa',
  'empenho',
  'valor_inicial_nc',
  'valor_utilizado',
  'valor_economizado',
  'saldo',
  'consumo_da_nc_%'],
 ['mes', 'indice_de_correcao', 'taxa_selic', 'observacao'],
 ['prop_pk1',
  'uf',
  'partido',
  'orgao',
  'prop_1',
  'prop_2',
  'prop_3',
  'prop_4'],
 ['processo',
  'proponente',
  'contrato',
  'nota_de_empenho',
  'valor_empenhado',
  'valor_pago',
  'saldo_do_empenho',
  'consumo_%',
  'janeiro',
  'fevereiro',
  'marco',
  'abril',
  'maio',
  'junho',
  'julho',
  'agosto',
  'setembro',
  'outubro',
  'novembro',
  'dezembro',
  'tipo']]

In [187]:
start_time = time()

#fazendo o loop sobre a lista acima para aplicar as funções que normalizam a coluna com as chaves dos proponentes
lst_df = [df_sn, df_an, df_cg]
for df in lst_df: 
    df['proponente'] = df['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    df['proponente'] = df['proponente'].apply(lambda x:str(x).strip())
#
lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])
#df_sn['proponente'] = df_sn['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
#df_sn['proponente'] = df_sn['proponente'].apply(lambda x:str(x).strip())
for cln in lst_pr:
    cln = cln.apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 1 seconds to run.


O objetivo inicial do projeto foi limpar os dados relacionados ao os proponentes com a criação de uma coluna nova referente ao nome civil do proponente para que esses valores pudessem ser usados como chave entre diferentes tabelas que serao parametrizadas entre si.

In [188]:
start_time = time()

#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''
    
#criando lista das colunas referente ao nome dos proponentes 
def filter_key(x):
    return (x == lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])

for df in lst_df:
    lst_prop_pk1 = []
    lst_prop_pk2= []
    for index,value in enumerate(df.proponente):
        filt_finding_key =  filter_key(df.proponente.iloc[index])
        result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
        result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
        lst_prop_pk1.append(result1)
        lst_prop_pk2.append(result2)
    df['prop_pk1'] = lst_prop_pk1
    df['prop_pk2'] = lst_prop_pk2

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 3 seconds to run.


In [189]:
#cleaning analitic sintetic rows

lst = ['SUBTOTAL','TOTAL','NAN']

for value in lst:
    df_sn = df_sn.loc[df_sn['proponente'] != value ]  
    
df_sn.fillna(0,inplace=True)
df_an.fillna(0,inplace=True)

In [190]:
dfs = [df_an, df_ic]

for index,df in enumerate(dfs): 
    meses = [mes for mes in df.mes.unique()]
    n_mes = ['01/2021','02/2021','03/2021','04/2021','05/2021','06/2021','07/2021','08/2021','09/2021','10/2021','11/2021','12/2021']
    meses = list(zip(meses, n_mes))

    for index,value in enumerate(meses):
        df['mes'].replace(meses[index][0],meses[index][1],inplace=True)


    df['mes']= pd.to_datetime(df['mes'])
    if (df.mes.dtype == '<M8[ns]') == True:
        print(f'The loop is ok!')
        
    else: print('The loops is broke!')
    
#df_an['mes'] = pd.to_datetime(df_an['mes'])

The loop is ok!
The loop is ok!


In [191]:
#checando se tem dfs sem prop_pk 
start_time = time()

dfs  = [df_an, df_sn]
lst_no_cv_name = []
for df in dfs:
    for index, value in enumerate(df.loc[df.prop_pk1==''].proponente.values):
        if value not in lst_no_cv_name:
            lst_no_cv_name.append(df.loc[df.prop_pk1==''].proponente.values[index])
        else:pass

print('DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:')
df_pr_nokey = pd.DataFrame()
df_pr_nokey['prop_nokey']= lst_no_cv_name
df_pr_nokey['prop_pk1']= ''
df_pr_nokey['orgao']=''
df_pr_nokey['uf']=''
df_pr_nokey['partido']=''

save = False
if save == True:
    df_pr_nokey.to_csv('ignore_prop_with_no_key.csv')
else:pass

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
df_pr_nokey

DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:
Time: This code took 1 seconds to run.


,prop_nokey,prop_pk1,orgao,uf,partido


In [192]:
start_time = time()

#installing the python library to save the data an excel with all the sheets
!pip install xlsxwriter

dfs=[]

#append dfs as str from the dictionary of data to a new list
for i,v in enumerate(df_nm):
    dfs.append(df_nm[i][0][0]) 

command = str('dfs = ' + str(dfs).replace("'",'')) # building a command with strings

print('This is the command >>>',command)

exec(command) # executing the str as a command

#giving the name to the excel sheets 
for i,df in enumerate(dfs):
  dfs[i].name = df_nm[i][0][1]

#creating the excel file
writer = pd.ExcelWriter('BI_Financeiro.xlsx',engine='xlsxwriter')
workbook=writer.book

#putting all dfs inside the excel
for df in dfs:
  worksheet=workbook.add_worksheet(df.name)
  writer.sheets[df.name] = worksheet
  df.to_excel(writer,sheet_name=df.name,startrow=0 , startcol=0, index=False)

#saving
writer.save()

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This code took 1 seconds to run.

Defaulting to user installation because normal site-packages is not writeable
This is the command >>> dfs = [df_an, df_sn, df_ic, df_pr, df_cg]
Time: This code took 3 seconds to run.


In [193]:
final_time= str(timedelta(seconds= total_time)); print(f'Time: The total time to run all the notebook was {final_time}.')
#Time: The total time to run all the notebook is 00:39 minutes.

Time: The total time to run all the notebook was 0:00:04.162071.
